In [23]:
# This program is designed to generate heat maps for S1 from a folder of .csv files.


# This cell is composed of the necessary imports and class formations for the program.

# Used for math opperations:
import numpy as np
import math

# Used for plotting data:
import matplotlib.pyplot as plt

# Used to read files:
import pandas as pd

In [15]:
'''This cell extracts a list of file names present in a folder, given the computer address of the folder. 
We need to draw a distinction between Windows devices and Mac/Linux devices because of the formatting
of addresses. When given the address of the folder, getFile returns the name of the file in the folder associated 
to a given n and s. getFiles iterates over a list 'nVals' of desired n values as strings, and a list 'sVals' of
desired s values as strings, and returns a list of the corresponding files.'''

def getFile(address: str, n: str, s: str) -> str:
    name = address + '/' + n + '_' + s + '.csv'
    return name


def getFiles(address: str, nVals: list[str], sVals: list[str]) -> list[str]:
    files = [getFile(address, n, s) for n in nVals for s in sVals]
    return files

def getFileWindows(address: str, n: str, s: str) -> str:
    name = address + '\\' + n + '_' + s + '.csv'
    return name


def getFilesWindows(address: str, nVals: list[str], sVals: list[str]) -> list[str]:
    files = [getFileWindows(address, n, s) for n in nVals for s in sVals]
    return files

In [22]:
'''This cell retrieves the data frames from the files in the folder. getDataFrame is able to obtain the data
frame of a given file, and getDataFrames iterates over all files obtained by getfiles and returns a list of
the corresponding datarames. getAllTrialData then uses this list to produce a list of the 'datum' columns 
of these data frames, converted into lists form.'''


def getDataFrame(file: str) -> pd.core.frame.DataFrame:
    colnames = ['k','datum']
    try: 
        fileData = pd.read_csv(file, names=colnames)
    except: 
        print("{} does not exist".format(file))
    return fileData

def getDataFrames(files: list[str]) -> list[pd.core.frame.DataFrame]:
    allData = [getDataFrame(file) for file in files]
    return allData


def getAllTrialData(files: list[str]) -> list[list[int]]:
    allTrialData = [list(dataFrame['datum']) for dataFrame in getDataFrames(files)]
    for trialData in allTrialData:
        trialData.append(0)
    return allTrialData

In [18]:
'''This cell creates functions that will produce thetas and a line of zeroes for a given n. The line will be 
used as the z-axis of the circle plots of the heat maps to ensure that they lie on the plane z = 0 for reference.
The thetas will be used to parametrize the scatterplot on the circle.'''

def getThetas(n):
    thetas = [(2*np.pi*k)/n for k in range(0, n+1)]
    return thetas

def getLine(n):
    line = [0 for k in range(n+1)]
    return line

In [19]:
# This cell creates a heat maps for each specified s for a fixed n.

def graphHeatMap(n: str, trialData: list[list[int]], sVals: list[str], nVals: list[str]) -> None:
    index = nVals.index(n) # This will allow us to ensure that we get the appropriate thetas and line for n.
    thetas = getThetas(int(nVals[index])) # Obtains the thetas.
    line = getLine(int(nVals[index])) # Obtains tthe line.
    fig = plt.figure(figsize=(30,10*math.ceil(len(sVals)/3))) # Creates a plot.
    fig.suptitle("N = {}".format(n), fontsize=15) # Labes the heat maps according the n value.
    # Creates an axis for each of the s values:
    axes = [plt.subplot(math.ceil(len(sVals)/3), 3, 1 + i, projection='3d') for i in range(len(sVals))]
    # Creates the heat maps:
    for i in range(len(sVals)):
        # Plots a black circle with the x- and y-values sin(thetas) and cos(thetas), and z = 0:
        axes[i].plot3D(np.cos(thetas), np.sin(thetas), line, alpha = .75, c = 'black')
        # Plots a 'coolwarm' heat map on the black circle according to the values of trialData[index*len(sVals)]:
        axes[i].scatter3D(np.cos(thetas), np.sin(thetas), line, cmap = 'coolwarm', c = trialData[index*len(sVals) + i], alpha = 1, s = 100)
        # Labels the graph according to the s-value:
        axes[i].set_title('s = {}'.format(sVals[i]))

In [20]:
# This cell creates all of the heat maps for all s and n specified. It runs graphHeatMap for each n specified.

def graphHeatMaps(trialData: list[list[int]], nVals: list[str], sVals: list[str]) -> None:
    for n in nVals:
        graphHeatMap(n, trialData, sVals, nVals)

In [13]:
# address = '/Users/alexanderneuschotz/Downloads/data2'
# nVals = ['10', '100', '10000']
# sVals = ['0.000000', '0.225000', '0.250000', '0.500000']
# files = getFiles(address, nVals, sVals)
# trialData = getAllTrialData(files)
# graphHeatMaps(trialData, nVals, sVals)